In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

from eegsc.ml.gru import GRUNet, train_gru
from eegsc.preprocessing.filters import BandPassFilter
from eegsc.utils.io import read_raw
from eegsc.utils.path import get_data_path
from eegsc.utils.experiments import create_sequence_dataset

/home/owerbat/miniconda3/envs/eeg/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = read_raw(os.path.join(get_data_path(), 'raw', '1st_Day.mat'),
                # data_types=['left_real', 'right_real',
                #             'left_im1', 'right_im1',
                #             'left_im2', 'right_im2',
                #             'left_quasi', 'right_quasi'])
                # data_types=['left_quasi', 'right_quasi'])
                # data_types=['left_im2', 'right_im2'])
                # data_types=['left_im1', 'right_im1'])
                data_types=['left_real', 'right_real'])

In [3]:
bandpass_filter = BandPassFilter()
seq_data, labels = create_sequence_dataset(data, bandpass_filter)
seq_data.shape, labels.shape

((569, 32, 10095), (569,))

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    seq_data, labels, test_size=0.2, random_state=0)
x_train.shape, x_test.shape

((455, 32, 10095), (114, 32, 10095))

In [5]:
model = GRUNet(input_size=x_train.shape[1],
               hidden_size=128,
               n_layers=1,
               n_classes=np.unique(labels).shape[0])

In [6]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.001)
model = train_gru(model, x_train, y_train, criterion, optimizer, n_epochs=1)

Epoch 1 | loss = 317.14584171772003, accuracy = 0.46593406593406594
